# A single Perceptron
We are using [this](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) data.
This is code to implement a single perceptron in Object Oriented Programming using the data from the coursework. 

### Imports & Data Setup
Exploratory Data Analysis and explanation for preprocessing can be found in perceptron.ipynb.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Loading the pre-processed data. Preprocessing can be found in perceptron.ipynb.
X = np.load("data/X-data.npy")
y = np.load("data/y-data.npy")

# Setting a random seed for consistency. 
np.random.seed(42)

### Object-Oriented Approach
#### Layers
Each layer in an ANN is made up of a number of neurons. Each of those neurons is just a visualization to make it easier for humans to understand how a neural network works.  
In practice, what you need for a layer on a very basic level is:
- an array of weights
- an array of biases
- an array of outputs
The length of all three arrays will be the same and depict the **number of neurons** on this specific layer.  
  
Then we also need an input for the layer. For the first layer, that will be our data. For the hidden layers and the output layer, that will be the output from the layer before. The input will need to be passed to the layer.  
  
Each layer must be able to process `activation_function(inputs * weights + bias)` and assign that to `layer.outputs`. 

## Class definitions

#### Constants

In [38]:
# Global variables.
learning_rate = 0.1
epochs = 100
np.random.seed(10)

In [76]:

class Layer:
    def __init__(self, nr_inputs, nr_neurons, activation_function):  # Constructor.
        self.weights = np.zeros(nr_inputs)  # Create random initial weights.
        self.biases = np.random.randn(nr_neurons)  # Create random initial biases.
        self.activation = activation_function  # I love how you can pass functions in python. 
        
        # Variables used for Backpropagation.
        
        
        self.output = []  # To save the outputs of this layer.
     
    # For pretty printing.   
    def __str__(self):
        return "Weights: "+ str(self.weights) + ",\n Biases: " + str(self.biases) + "\nActivation: "+ str(self.activation)
        
    def forward_propagation(self, inputs):
        self.output.append(self.activation(np.dot(inputs, self.weights) + self.biases)[0])
        
    def back_propagation(self, y, y_hat):
        self.weights += learning_rate * (y - y_hat) * y_hat
        self.biases += learning_rate * (y - y_hat)

#### Activation Functions
Then we need a collection of different activation functions, as each of our layers will have a specific activation function.

In [77]:
class ActivationFunctions:
    def __init__(self, step_threshold=0.5): 
        self.step_threshold = step_threshold  # Use a default threshold of 0.5.
        
    def relu(x):
        return np.maximum(x, 0.0)
    
    def step(self, x):
        for row, element in enumerate(x):
            x[row] = 1.0 if element > self.step_threshold else 0.0
        return x
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

## Training 

In [84]:
step_function = ActivationFunctions().step
perceptron = Layer(X.shape[1], 10, step_function)

for _ in range(1):
    for index, x in enumerate(X):
        perceptron.forward_propagation(x)
        perceptron.back_propagation(y[index], perceptron.output[index])

In [86]:
print(perceptron.__str__())

Weights: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1.],
 Biases: [5.64381452 7.2772263  7.43415215 5.36828789 7.45116949 7.89334122
 7.71726515 5.50560623 8.60770823 7.74454398]
Activation: <bound method ActivationFunctions.step of <__main__.ActivationFunctions object at 0x000001E87E25A580>>


## Evaluation

In [87]:
res = pd.DataFrame()
res['Predictions'] = perceptron.output
res['Predictions'] = res['Predictions'].apply(lambda x: 0 if x < 0.5 else 1)
res['Expectation'] = y

print("Accuracy: ", res.loc[res['Predictions']==res['Expectation']].shape[0] / res.shape[0] * 100)

Accuracy:  83.30404217926186
